# PPO training for Task 7 pendulum

Reproduces the PPO workflow from `task_2_lqr_balance_data_gen.ipynb`, but runs directly on the custom PyBullet + Pinocchio environment implemented in `scripts/task_7_env.py`.

In [1]:
import os
from pathlib import Path

import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
current = Path.cwd()

if (current / 'notebooks').exists():
    PROJECT_ROOT = current
else:
    PROJECT_ROOT = current.parent

os.chdir(PROJECT_ROOT)

import sys
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from scripts.task_7_env import Task7PendulumEnv

DATA_DIR = Path('data')
MODELS_DIR = DATA_DIR
for directory in (DATA_DIR, MODELS_DIR):
    directory.mkdir(exist_ok=True)

print('Project root:', PROJECT_ROOT)

Project root: /home/acepeax/Desktop/Studies/MVA/Robotics/Project


pybullet build time: Dec  4 2025 20:11:42


## Environment helpers

In [3]:
MAX_STEPS = 600
SHOULD_BALANCE = True
SIM_SUBSTEPS = 12

def make_task7_env(gui=False, should_balance=SHOULD_BALANCE):
    def _init():
        env = Task7PendulumEnv(
            max_steps=MAX_STEPS,
            should_balance=should_balance,
            gui=gui,
            sim_substeps=SIM_SUBSTEPS,
        )
        return Monitor(env)
    return _init

train_env = DummyVecEnv([make_task7_env(gui=False, should_balance=SHOULD_BALANCE)])
print('Observation space:', train_env.observation_space)
print('Action space:', train_env.action_space)

Observation space: Box(-inf, inf, (14,), float32)
Action space: Discrete(3)


## Train PPO

In [ ]:
model = PPO(
    policy='MlpPolicy',
    env=train_env,
    verbose=0,
    n_steps=2048,
    batch_size=256,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.0,
    learning_rate=3e-4,
    clip_range=0.2,
    tensorboard_log='runs/task7_ppo',
)

Using cpu device


In [5]:
TOTAL_TIMESTEPS = 200_000
model.learn(total_timesteps=TOTAL_TIMESTEPS, progress_bar=True)

Logging to runs/task7_ppo/PPO_1


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.84     |
|    ep_rew_mean     | 4.84     |
| time/              |          |
|    fps             | 274      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.6         |
|    ep_rew_mean          | 4.6         |
| time/                   |             |
|    fps                  | 268         |
|    iterations           | 2           |
|    time_elapsed         | 15          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011344364 |
|    clip_fraction        | 0.187       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.00491    |
|    learning_rate        | 0.0003      |
|    loss                 | 3.77        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0272     |
|    value_loss           | 12.2        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.89        |
|    ep_rew_mean          | 6.89        |
| time/                   |             |
|    fps                  | 261         |
|    iterations           | 3           |
|    time_elapsed         | 23          |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.019952781 |
|    clip_fraction        | 0.275       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.05       |
|    explained_variance   | 0.0684      |
|    learning_rate        | 0.0003      |
|    loss                 | 2.96        |
|    n_updates            | 20          |
|    policy_gradient_loss | -0.0348     |
|    value_loss           | 7.07        |
-----------------------------------------


...

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 509          |
|    ep_rew_mean          | 509          |
| time/                   |              |
|    fps                  | 200          |
|    iterations           | 94           |
|    time_elapsed         | 960          |
|    total_timesteps      | 192512       |
| train/                  |              |
|    approx_kl            | 0.0022355276 |
|    clip_fraction        | 0.0222       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.22        |
|    explained_variance   | 0.0123       |
|    learning_rate        | 0.0003       |
|    loss                 | 72.1         |
|    n_updates            | 930          |
|    policy_gradient_loss | -0.0027      |
|    value_loss           | 149          |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 512          |
|    ep_rew_mean          | 512          |
| time/                   |              |
|    fps                  | 200          |
|    iterations           | 95           |
|    time_elapsed         | 971          |
|    total_timesteps      | 194560       |
| train/                  |              |
|    approx_kl            | 0.0016831752 |
|    clip_fraction        | 0.00688      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.216       |
|    explained_variance   | 0.363        |
|    learning_rate        | 0.0003       |
|    loss                 | 50.1         |
|    n_updates            | 940          |
|    policy_gradient_loss | -0.0017      |
|    value_loss           | 115          |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 512          |
|    ep_rew_mean          | 512          |
| time/                   |              |
|    fps                  | 200          |
|    iterations           | 96           |
|    time_elapsed         | 982          |
|    total_timesteps      | 196608       |
| train/                  |              |
|    approx_kl            | 0.0008507465 |
|    clip_fraction        | 0.0214       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.21        |
|    explained_variance   | 0.406        |
|    learning_rate        | 0.0003       |
|    loss                 | 59.2         |
|    n_updates            | 950          |
|    policy_gradient_loss | -0.00251     |
|    value_loss           | 122          |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 512          |
|    ep_rew_mean          | 512          |
| time/                   |              |
|    fps                  | 200          |
|    iterations           | 97           |
|    time_elapsed         | 992          |
|    total_timesteps      | 198656       |
| train/                  |              |
|    approx_kl            | 0.0014845565 |
|    clip_fraction        | 0.0249       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.217       |
|    explained_variance   | -0.0446      |
|    learning_rate        | 0.0003       |
|    loss                 | 54.4         |
|    n_updates            | 960          |
|    policy_gradient_loss | -0.00165     |
|    value_loss           | 91.5         |
------------------------------------------


------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 514          |
|    ep_rew_mean          | 514          |
| time/                   |              |
|    fps                  | 199          |
|    iterations           | 98           |
|    time_elapsed         | 1004         |
|    total_timesteps      | 200704       |
| train/                  |              |
|    approx_kl            | 0.0028758259 |
|    clip_fraction        | 0.0268       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.22        |
|    explained_variance   | -0.0142      |
|    learning_rate        | 0.0003       |
|    loss                 | 34           |
|    n_updates            | 970          |
|    policy_gradient_loss | -0.00158     |
|    value_loss           | 116          |
------------------------------------------


## Save / load the trained policy

In [ ]:
MODEL_PATH = MODELS_DIR / 'test_ppo_task7_balance'
model.save(MODEL_PATH)
print('Model saved to', MODEL_PATH)

Model saved to data/ppo_task7_balance


In [4]:
MODEL_PATH = MODELS_DIR / 'ppo_task7_balance'
model = PPO.load(MODEL_PATH, train_env)

## Rollout helpers

In [5]:
train_env.close()

def rollout_episode(env, model, max_steps=MAX_STEPS, deterministic=True):
    obs, _ = env.reset()
    rewards = []
    infos = []
    for _ in range(max_steps):
        action, _ = model.predict(obs, deterministic=deterministic)
        obs, reward, terminated, truncated, info = env.step(action)
        rewards.append(reward)
        infos.append(info)
        if terminated or truncated:
            break
    return np.array(rewards), infos

eval_env = Task7PendulumEnv(
    max_steps=MAX_STEPS, should_balance=SHOULD_BALANCE, gui=True, sim_substeps=SIM_SUBSTEPS
)

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Intel
GL_RENDERER=Mesa Intel(R) Iris(R) Xe Graphics (RPL-U)
GL_VERSION=4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.1
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 25.0.7-0ubuntu0.24.04.1
Vendor = Intel
Renderer = Mesa Intel(R) Iris(R) Xe Graphics (RPL-U)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubuntu
ven = Intel
Workaround for some crash in the Intel OpenGL driver on Linux/Ubunt

In [9]:
rewards, infos = rollout_episode(eval_env, model)
print('Episode length:', len(rewards))
print('Total reward:', rewards.sum())
print('Final info:', infos[-1] if infos else None)

KeyboardInterrupt: 

## Export rollouts for datasets

In [ ]:
def record_task7_dataset(
    model,
    n_episodes=5,
    max_steps=MAX_STEPS,
    should_balance=SHOULD_BALANCE,
    filename='task7_ppo_dataset',
):
    env = Task7PendulumEnv(
        max_steps=max_steps, should_balance=should_balance, gui=False, sim_substeps=SIM_SUBSTEPS
    )
    all_obs, all_actions, all_rewards, episode_ids = [], [], [], []
    recorded = 0
    while recorded < n_episodes:
        obs, _ = env.reset()
        local_obs, local_actions, local_rewards = [], [], []
        success = True
        for step in range(max_steps):
            local_obs.append(obs.copy())
            action, _ = model.predict(obs, deterministic=True)
            action = int(action)
            local_actions.append(action)
            obs, reward, terminated, truncated, info = env.step(action)
            local_rewards.append(reward)
            if terminated or truncated:
                success = info.get('success', False) and not info.get('failure', False)
                break
        if success and len(local_obs) == max_steps:
            all_obs.extend(local_obs)
            all_actions.extend(local_actions)
            all_rewards.extend(local_rewards)
            episode_ids.extend([recorded] * len(local_obs))
            recorded += 1
            print(f'Recorded episode {recorded}/{n_episodes}')
        else:
            print('Episode failed, retrying...')
    env.close()
    save_path = DATA_DIR / f'{filename}.npz'
    np.savez(
        save_path,
        observations=np.array(all_obs, dtype=np.float32),
        actions=np.array(all_actions, dtype=np.int64),
        rewards=np.array(all_rewards, dtype=np.float32),
        episode_ids=np.array(episode_ids, dtype=np.int32),
        max_steps=max_steps,
    )
    print('Dataset saved to', save_path)
    return save_path

In [ ]:
# Example usage once a good policy is trained
# dataset_path = record_task7_dataset(model, n_episodes=3, filename='task7_ppo_balance_rollouts')

In [ ]:
eval_env.close()
train_env.close()